In [46]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import nltk
nltk.download('stopwords')
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
#Load the dataset and explore.
try:
    from google.colab import files
    !wget -P DATAPATH https://raw.githubusercontent.com/practical-nlp/practical-nlp/master/Ch4/Data/Sentiment%20and%20Emotion%20in%20Text/train_data.csv
    !wget -P DATAPATH https://raw.githubusercontent.com/practical-nlp/practical-nlp/master/Ch4/Data/Sentiment%20and%20Emotion%20in%20Text/test_data.csv
    !ls -lah DATAPATH
    filepath = "DATAPATH/train_data.csv"
except ModuleNotFoundError:
    filepath = "Data/Sentiment and Emotion in Text/train_data.csv"

--2022-04-13 08:42:21--  https://raw.githubusercontent.com/practical-nlp/practical-nlp/master/Ch4/Data/Sentiment%20and%20Emotion%20in%20Text/train_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2479133 (2.4M) [text/plain]
Saving to: ‘DATAPATH/train_data.csv.1’

train_data.csv.1    100%[===================>]   2.36M  --.-KB/s    in 0.06s   

2022-04-13 08:42:21 (36.7 MB/s) - ‘DATAPATH/train_data.csv.1’ saved [2479133/2479133]

--2022-04-13 08:42:21--  https://raw.githubusercontent.com/practical-nlp/practical-nlp/master/Ch4/Data/Sentiment%20and%20Emotion%20in%20Text/test_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.github

In [4]:
data = pd.read_csv(filepath)
print(data.shape)
data.head()

(30000, 2)


,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...


In [5]:
data['sentiment'].value_counts()

worry         7433
neutral       6340
sadness       4828
happiness     2986
love          2068
surprise      1613
hate          1187
fun           1088
relief        1021
empty          659
enthusiasm     522
boredom        157
anger           98
Name: sentiment, dtype: int64

In [9]:
#taking only the top best categories
top_list = ['worry','neutral','sadness']
df_subset = data[data['sentiment'].isin(top_list)]
print(df_subset.shape)

(18601, 2)


In [11]:
content = df_subset['content'].tolist()
content

['Layin n bed with a headache  ughhhh...waitin on your call...',
 'Funeral ceremony...gloomy friday...',
 '@dannycastillo We want to trade with someone who has Houston tickets, but no one will.',
 "Re-pinging @ghostridah14: why didn't you go to prom? BC my bf didn't like my friends",
 "I should be sleep, but im not! thinking about an old friend who I want. but he's married now. damn, &amp; he wants me 2! scandalous!",
 'Hmmm. http://www.djhero.com/ is down',
 '@charviray Charlene my love. I miss you',
 "@kelcouch I'm sorry  at least it's Friday?",
 'cant fall asleep',
 'Choked on her retainers',
 'Ugh! I have to beat this stupid song to get to the next  rude!',
 '@BrodyJenner if u watch the hills in london u will realise what tourture it is because were weeks and weeks late  i just watch itonlinelol',
 'The storm is here and the electricity is gone',
 "So sleepy again and it's not even that late. I fail once again.",
 '@PerezHilton lady gaga tweeted about not being impressed by her vid

In [34]:
#strip_handles removes personal information such as twitter handles, which don't
#contribute to emotion in the tweet. preserve_case=False converts everything to lowercase.
tweeter = TweetTokenizer(strip_handles= True, preserve_case = False )
mystopwords = set(stopwords.words("english"))

#Function to tokenize tweets, remove stopwords and numbers. 
#Keeping punctuations and emoticon symbols could be relevant for this task!
def preprocess_corpus(text):

  #Nested function that removes stopwords and digits from a list of tokens
  def remove_stops_digits(tokens):
    return [token for token in tokens if token not in mystopwords and not token.isdigit()]
  
  return [remove_stops_digits(tweeter.tokenize(content)) for content in text]


#setting on actual data
my_data = preprocess_corpus(df_subset['content'].tolist())
my_cats = df_subset['sentiment']
print(len(my_data),len(my_cats))

18601 18601


In [79]:
#Split data into train and test, following the usual process
train_data, test_data, train_cats, test_cats = train_test_split(my_data,my_cats, random_state=1224)
#prepare training data in doc2vec format:
train_doc2vec = [TaggedDocument((d), tags = [str(i)]) for i , d in enumerate(train_data)]
#Train a doc2vec model to learn tweet representations. Use only training data!!
model = Doc2Vec(vector_size = 50, alpha = 0.025, min_count = 5, dm = 1, epochs = 100,)
model.build_vocab(train_doc2vec)
model.train(train_doc2vec, total_examples = model.corpus_count, epochs = model.epochs)
model.save("d2v.model")
print("Model Saved")

Model Saved


In [62]:
#Infer the feature representation for training and test data using the trained model
model = Doc2Vec.load("d2v.model")
#infer in multiple steps to get a stable representation. 
train_vectors = [model.infer_vector(list_of_tokens, steps = 50) for list_of_tokens in train_data]
test_vectors = [model.infer_vector(list_of_tokens, steps = 50) for list_of_tokens in test_data]

In [67]:
#train_vectors

### Modelling

In [48]:
#predicting using logistic regression
my_model = LogisticRegression(class_weight = 'balanced')
my_model.fit(train_vectors,train_cats)
print(accuracy_score(test_cats, my_model.predict(test_vectors)))

0.41775962158675556
